<a href="https://colab.research.google.com/gist/plasmax/6e1c8db32589247aee1b9783641e8b00/dav3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Depth Anything V3 - Complete Testing Notebook

This notebook demonstrates:
- Setting up Depth Anything V3
- Caching models to Google Drive for faster re-runs
- Processing video files
- Generating depth maps
- Creating 3D Gaussians for novel view synthesis
- Exporting results in multiple formats

## 1. Setup - Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Path to your video file
path = '/content/drive/MyDrive/Research/Assets/IMG_1519.MOV'

## 2. Create Working Directory

In [ ]:
import os

# Define the target directory in Google Drive
target_dir = '/content/drive/MyDrive/Research/Depth/DAv3'

# Create the directory if it doesn't exist
os.makedirs(target_dir, exist_ok=True)
print(f"Ensured directory exists: {target_dir}")

# Change the current working directory to the target directory
os.chdir(target_dir)
print(f"Changed current working directory to: {os.getcwd()}")

## 3. Clone Repository

In [ ]:
# Clone the repository
!git clone https://github.com/ByteDance-Seed/Depth-Anything-3 .
print("Repository cloned successfully!")

The repository has been cloned into `/content/drive/MyDrive/Research/Depth/DAv3`.

## 4. Install Dependencies

Installing Depth Anything 3 with Gaussian Splatting support. This may take several minutes.

In [ ]:
# Install the package with Gaussian support
# This includes gsplat for 3D Gaussian Splatting
!pip install -e ".[gs]" --quiet
print("Installation complete!")

## 5. Verify Installation

In [ ]:
# Verify installation and check GPU
import torch
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"CUDA version: {torch.version.cuda}")
    print(f"GPU device: {torch.cuda.get_device_name(0)}")
    print(f"GPU memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB")

## 6. Download and Cache Model

Download the DA3NESTED-GIANT-LARGE model to Google Drive. This will be saved permanently in `checkpoints/` so you don't need to re-download it in future sessions.

In [ ]:
import os
from huggingface_hub import snapshot_download

# Setup checkpoints directory in Google Drive
checkpoints_dir = '/content/drive/MyDrive/Research/Depth/DAv3/checkpoints'
os.makedirs(checkpoints_dir, exist_ok=True)
print(f"Checkpoints directory: {checkpoints_dir}")

# Define model path
model_name = "DA3NESTED-GIANT-LARGE"
model_path = os.path.join(checkpoints_dir, model_name)

# Download model if not already cached
if os.path.exists(model_path) and os.listdir(model_path):
    print(f"\n✓ Model already exists at: {model_path}")
    print("Skipping download (using cached model)")
else:
    print(f"\nDownloading {model_name} to Google Drive...")
    print("This may take several minutes on first run...")
    snapshot_download(
        repo_id=f"depth-anything/{model_name}",
        local_dir=model_path,
        local_dir_use_symlinks=False
    )
    print(f"\n✓ Model downloaded successfully to: {model_path}")

# Verify model files
model_files = os.listdir(model_path)
print(f"\nModel contains {len(model_files)} files")
total_size = sum(os.path.getsize(os.path.join(model_path, f)) for f in model_files if os.path.isfile(os.path.join(model_path, f)))
print(f"Total size: {total_size / 1e9:.2f} GB")

## 7. Process Video with Depth Anything V3

This will:
- Extract frames from your video at 2 FPS
- Estimate depth for each frame
- Estimate camera poses
- Generate 3D Gaussians
- Create a GLB file for 3D visualization
- Generate a Gaussian Splatting video for novel view synthesis

In [ ]:
# Setup output directory
output_dir = '/content/drive/MyDrive/Research/Depth/DAv3/output/IMG_1519'
os.makedirs(output_dir, exist_ok=True)
print(f"Output directory: {output_dir}")

In [ ]:
# Process the video with Gaussian generation
# Using the locally cached DA3NESTED-GIANT-LARGE model from Google Drive
!da3 video {path} \
    --model-dir {model_path} \
    --export-dir {output_dir} \
    --export-format glb-gs_video \
    --fps 2.0 \
    --process-res 504 \
    --show-cameras True \
    --auto-cleanup

print("\n" + "="*50)
print("Processing complete!")
print("="*50)

## 8. Check Output Files

In [ ]:
# List all generated files
import os

print("Generated files:")
print("="*50)
for root, dirs, files in os.walk(output_dir):
    level = root.replace(output_dir, '').count(os.sep)
    indent = ' ' * 2 * level
    print(f"{indent}{os.path.basename(root)}/")
    subindent = ' ' * 2 * (level + 1)
    for file in files:
        file_path = os.path.join(root, file)
        file_size = os.path.getsize(file_path) / (1024 * 1024)  # Size in MB
        print(f"{subindent}{file} ({file_size:.2f} MB)")

## 9. Visualize Results

Display the thumbnail and Gaussian Splatting video

In [ ]:
# Display the scene thumbnail
from IPython.display import Image, display
import os

thumbnail_path = os.path.join(output_dir, 'scene.jpg')
if os.path.exists(thumbnail_path):
    print("Scene Thumbnail:")
    display(Image(filename=thumbnail_path, width=600))
else:
    print("Thumbnail not found")

In [ ]:
# Display Gaussian Splatting video if available
from IPython.display import Video, display
import glob

gs_video_dir = os.path.join(output_dir, 'gs_video')
if os.path.exists(gs_video_dir):
    videos = glob.glob(os.path.join(gs_video_dir, '*.mp4'))
    if videos:
        print(f"Found {len(videos)} Gaussian Splatting video(s)")
        for video_path in videos:
            print(f"\nShowing: {os.path.basename(video_path)}")
            display(Video(video_path, width=600))
    else:
        print("No videos found in gs_video directory")
else:
    print("Gaussian video directory not found")

## 10. View 3D Scene (GLB)

Download the `scene.glb` file and view it in:
- [Babylon.js Sandbox](https://sandbox.babylonjs.com/)
- [glTF Viewer](https://gltf-viewer.donmccurdy.com/)
- Any 3D viewer that supports GLB format

In [ ]:
# Provide download link for GLB file
from google.colab import files
import os

glb_path = os.path.join(output_dir, 'scene.glb')
if os.path.exists(glb_path):
    file_size = os.path.getsize(glb_path) / (1024 * 1024)
    print(f"GLB file size: {file_size:.2f} MB")
    print(f"GLB file location: {glb_path}")
    print("\nTo download, uncomment and run the following line:")
    print(f"# files.download('{glb_path}')")
else:
    print("GLB file not found")

## 11. (Optional) Use Python API Directly

For more control, you can use the Python API instead of CLI

In [ ]:
# Example: Using Python API for more control
import torch
from depth_anything_3.api import DepthAnything3
import glob
import os

# This is optional - only run if you want programmatic control
# Uncomment to use:

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# # Use the locally cached model from Google Drive
# model = DepthAnything3.from_pretrained(model_path)
# model = model.to(device=device)

# # Process images from a directory
# image_dir = "path/to/extracted/frames"
# images = sorted(glob.glob(os.path.join(image_dir, "*.png")))

# prediction = model.inference(
#     images,
#     export_dir="output_python_api",
#     export_format="glb-gs_video"
# )

# print(f"Depth shape: {prediction.depth.shape}")
# print(f"Extrinsics shape: {prediction.extrinsics.shape}")
# print(f"Intrinsics shape: {prediction.intrinsics.shape}")

print("Python API code ready (currently commented out)")

## Summary

This notebook has:
1. ✅ Installed Depth Anything V3 with Gaussian support
2. ✅ Downloaded and cached the model to Google Drive (`checkpoints/`)
3. ✅ Processed your video file (IMG_1519.MOV)
4. ✅ Generated depth maps
5. ✅ Estimated camera poses
6. ✅ Created 3D Gaussians
7. ✅ Exported results as GLB and Gaussian video

### Output Files:
- `scene.glb` - 3D scene with depth point cloud
- `scene.jpg` - Thumbnail visualization
- `gs_video/` - Gaussian Splatting videos for novel view synthesis

### Cached Model:
- **Location:** `/content/drive/MyDrive/Research/Depth/DAv3/checkpoints/DA3NESTED-GIANT-LARGE`
- **Benefit:** Model persists in Google Drive - no need to re-download in future sessions!

### Next Steps:
- Download and view the GLB file in a 3D viewer
- Watch the Gaussian Splatting videos above
- Experiment with different parameters (fps, process-res, etc.)
- Try different models (DA3-GIANT, DA3-LARGE, etc.)
- Re-run the notebook - it will use the cached model and start much faster!